In [23]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from joblib import load
import matplotlib.pyplot as plt

def get_candles(interval, prev_val = 0):
    tv = TvDatafeed()

    if interval == "1_minute":
        niftyIndex = tv.get_hist(symbol='NIFTY', exchange='NSE', interval=Interval.in_1_minute, n_bars=(211 + prev_val))
    elif interval == "3_minute":
        niftyIndex = tv.get_hist(symbol='NIFTY', exchange='NSE', interval=Interval.in_3_minute, n_bars=(211 + prev_val))
    elif interval == "5_minute":
        niftyIndex = tv.get_hist(symbol='NIFTY', exchange='NSE', interval=Interval.in_5_minute, n_bars=(211 + prev_val))
    elif interval == "15_minute":
        niftyIndex = tv.get_hist(symbol='NIFTY', exchange='NSE', interval=Interval.in_15_minute, n_bars=(211 + prev_val))
    elif interval == "1_hour":
        niftyIndex = tv.get_hist(symbol='NIFTY', exchange='NSE', interval=Interval.in_1_hour, n_bars=(211 + prev_val))
    else:
        raise Exception("Invalid Interval")

    data = pd.DataFrame(niftyIndex)
    data.reset_index(inplace=True)
    data.drop(columns=["symbol", "volume"], inplace=True)

    currentDate = pd.Timestamp.now()
    if data.iloc[-1]['datetime'] < currentDate:
        data = data.iloc[:-1]

    if prev_val != 0:
        data = data.iloc[:-prev_val].reset_index(drop=True)

    #print(data.tail)

    #--------------------------------------------------------------

    data["datetime"] = pd.to_datetime(data["datetime"])

    data['day'] = data['datetime'].dt.dayofweek
    data['hour'] = data['datetime'].dt.hour
    if interval != "1_hour":
        data['minute'] = data['datetime'].dt.minute

    data["mv20"] = data["close"].rolling(window=20).mean()
    data["mv50"] = data["close"].rolling(window=50).mean()
    data["mv200"] = data["close"].rolling(window=200).mean()

    data['tr'] = np.maximum(data['high'] - data['low'], np.abs(data['high'] - data['close'].shift()), np.abs(data['low'] - data['close'].shift()))
    data['atr'] = data['tr'].rolling(window=14).mean()
    data.drop(columns=['tr'], inplace=True)

    data = data.iloc[-11:].reset_index(drop=True)

    #print(data)

    #--------------------------------------------------------------

    columns_to_difference = ["open", "high", "low", "close", "mv20", "mv50", "mv200", "atr"]
    if interval != "1_hour":
        other_columns = ["datetime", "day", "hour", "minute"]
    else:
        other_columns = ["datetime", "day", "hour"]

    data_diff = pd.DataFrame()
    data_diff[other_columns] = data[other_columns]

    for col in columns_to_difference:
        data_diff[col] = data[col].diff()

    data_diff = data_diff.iloc[1:].reset_index(drop=True)

    #--------------------------------------------------------------

    target_candle_data_cols = ['high', 'low']
    if interval != "1_hour":
        other_candle_data_cols = ['day', 'hour', 'minute', 'open', 'close', 'mv20', 'mv50', 'mv200', 'atr']
        candle_data_cols = ['open', 'high', 'low', 'close', 'day', 'hour', 'minute', 'mv20', 'mv50', 'mv200', 'atr']
    else:
        other_candle_data_cols = ['day', 'hour', 'open', 'close', 'mv20', 'mv50', 'mv200', 'atr']
        candle_data_cols = ['open', 'high', 'low', 'close', 'day', 'hour', 'mv20', 'mv50', 'mv200', 'atr']

    scalar_candle = load(f'archive/scalars/scalar_{interval}_candle.save')
    scalar_candle_target = load(f'archive/scalars/scalar_{interval}_candle_target.save')

    data_diff[other_candle_data_cols] = scalar_candle.transform(data_diff[other_candle_data_cols])
    data_diff[target_candle_data_cols] = scalar_candle_target.transform(data_diff[target_candle_data_cols])
    #print(data_diff.head(15))

    #--------------------------------------------------------------

    PAST_WINDOW = 10

    candle_sequences = []
    for ind in range(len(data_diff) - PAST_WINDOW + 1):
        candle_seq = data_diff.iloc[ind : ind + PAST_WINDOW][candle_data_cols].values
        candle_sequences.append(candle_seq)

    candle_sequences = np.array(candle_sequences)

    return candle_sequences, data

In [24]:
def predict_n_prepare(candle_sequences, data, interval):
    data = data.iloc[1:].reset_index(drop=True)
    model = load(f"archive/models/model_{interval}.save")
    scaled_pred = model.predict(candle_sequences)
    
    scalar_candle_target = load(f'archive/scalars/scalar_{interval}_candle_target.save')
    y_pred = scalar_candle_target.inverse_transform(scaled_pred.reshape(-1, scaled_pred.shape[-1])).reshape(scaled_pred.shape)
    #print(y_pred[0][0])

    y_pred_flat = y_pred.reshape(-1, 2)
    last_row = data[["high", "low"]].iloc[-1]
    new_row = last_row + y_pred_flat[0]
    new_row = np.round(new_row, 2) 
    new_row_df = pd.DataFrame([new_row], columns=["high", "low"])
    if interval == "1_minute":
        new_row_df["datetime"] = pd.to_datetime(data["datetime"].iloc[-1]) + pd.Timedelta(minutes=1)
    elif interval == "3_minute":
        new_row_df["datetime"] = pd.to_datetime(data["datetime"].iloc[-1]) + pd.Timedelta(minutes=3)
    elif interval == "5_minute":
        new_row_df["datetime"] = pd.to_datetime(data["datetime"].iloc[-1]) + pd.Timedelta(minutes=5)
    elif interval == "15_minute":
        new_row_df["datetime"] = pd.to_datetime(data["datetime"].iloc[-1]) + pd.Timedelta(minutes=15)
    elif interval == "1_hour":
        new_row_df["datetime"] = pd.to_datetime(data["datetime"].iloc[-1]) + pd.Timedelta(minutes=60)

    plot_data = data[["datetime", "high", "low"]].copy()
    plot_data = pd.concat([plot_data, new_row_df], ignore_index=True)
    plot_data = plot_data.iloc[1:].reset_index(drop=True)
    return plot_data

def magic_plot(candle_sequences, data, model):
    scaled_pred = model.predict(candle_sequences)
    
    scalar_candle_target = load('archive/scalar_candle_target.save')
    y_pred = scalar_candle_target.inverse_transform(scaled_pred.reshape(-1, scaled_pred.shape[-1])).reshape(scaled_pred.shape)
    print(y_pred[0][0])

    y_pred_flat = y_pred.reshape(-1, 2)
    last_row = data[["high", "low"]].iloc[-1]
    new_row = last_row + y_pred_flat[0]
    new_row_df = pd.DataFrame([new_row], columns=["high", "low"])
    new_row_df["datetime"] = pd.to_datetime(data["datetime"].iloc[-1]) + pd.Timedelta(minutes=5)

    plot_data = data[["datetime", "high", "low"]].copy()
    plot_data = pd.concat([plot_data, new_row_df], ignore_index=True)
    #print(plot_data.head(15))
    
    plt.figure(figsize=(6, 4))
    plt.title("Magic Graph", fontsize=20)
    plt.plot(plot_data["datetime"], plot_data["high"], label="High", color="blue")
    plt.plot(plot_data["datetime"], plot_data["low"], label="Low", color="red")
    
    new_datetime = plot_data["datetime"].iloc[-1]
    new_high = plot_data["high"].iloc[-1]
    new_low = plot_data["low"].iloc[-1]

    plt.text(new_datetime, new_high, f"{new_high:.2f}", color="blue", fontsize=10, ha="center", va="bottom")
    plt.text(new_datetime, new_low, f"{new_low:.2f}", color="red", fontsize=10, ha="center", va="top")
    
    plt.legend()
    plt.grid()
    plt.xlabel("Datetime")
    plt.tight_layout()
    plt.show(block=False)
    plt.pause(0.1)

    cur_time = pd.Timestamp.now()
    if (cur_time.minute % 5) == 0 :
        print(cur_time.minute, 5 * 60 - abs(cur_time.second-2), "1ok")
        sleep(5 * 60 - abs(cur_time.second-2))
        plt.close()
    else:
        print(cur_time.minute, ((5-(cur_time.minute % 5)) * 60 - abs(cur_time.second-2)), "2ok")
        sleep((5-(cur_time.minute % 5)) * 60 - abs(cur_time.second-2))
        plt.close()




model = load("archive/model_only2-2.save")

"""for ind in range(10):
    candle_sequences, data = get_data(ind+1)
    magic_plot(candle_sequences, data, model)"""

candle_sequences, data = get_data()
magic_plot(candle_sequences, data, model)

while True:
    cur_time = pd.Timestamp.now()
    if (cur_time.minute % 5) == 0 :
        candle_sequences, data = get_data()
        magic_plot(candle_sequences, data, model)

In [25]:
candle_sequences, data = get_candles("1_hour")
print("Input shape",candle_sequences.shape)

Input shape (1, 10, 10)


In [26]:
plot_data = predict_n_prepare(candle_sequences, data, "1_hour")
print(plot_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
             datetime      high       low
0 2024-12-12 12:15:00  24583.65  24551.65
1 2024-12-12 13:15:00  24579.85  24543.05
2 2024-12-12 14:15:00  24577.05  24527.95
3 2024-12-12 15:15:00  24561.30  24534.60
4 2024-12-13 09:15:00  24520.80  24343.70
5 2024-12-13 10:15:00  24353.15  24180.80
6 2024-12-13 11:15:00  24598.80  24266.85
7 2024-12-13 12:15:00  24688.20  24497.90
8 2024-12-13 13:15:00  24757.90  24574.80
9 2024-12-13 14:15:00  24810.50  24584.67


In [27]:
candle_sequences, data = get_candles("5_minute")
print("Input shape",candle_sequences.shape)
plot_data = predict_n_prepare(candle_sequences, data, "5_minute")
print(plot_data)

Input shape (1, 10, 11)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
             datetime      high       low
0 2024-12-13 13:35:00  24707.70  24669.05
1 2024-12-13 13:40:00  24736.35  24693.15
2 2024-12-13 13:45:00  24748.65  24719.20
3 2024-12-13 13:50:00  24757.90  24720.55
4 2024-12-13 13:55:00  24739.90  24707.70
5 2024-12-13 14:00:00  24753.45  24729.65
6 2024-12-13 14:05:00  24743.25  24713.90
7 2024-12-13 14:10:00  24746.25  24722.70
8 2024-12-13 14:15:00  24749.55  24722.00
9 2024-12-13 14:20:00  24753.26  24735.89


In [28]:
plot_data["datetime"] = pd.to_datetime(plot_data["datetime"])
plot_data["datetime"] = plot_data["datetime"].dt.strftime("%H:%M")
json_output = plot_data.to_json(orient="records")
print(json_output)

[{"datetime":"13:35","high":24707.7,"low":24669.05},{"datetime":"13:40","high":24736.35,"low":24693.15},{"datetime":"13:45","high":24748.65,"low":24719.2},{"datetime":"13:50","high":24757.9,"low":24720.55},{"datetime":"13:55","high":24739.9,"low":24707.7},{"datetime":"14:00","high":24753.45,"low":24729.65},{"datetime":"14:05","high":24743.25,"low":24713.9},{"datetime":"14:10","high":24746.25,"low":24722.7},{"datetime":"14:15","high":24749.55,"low":24722.0},{"datetime":"14:20","high":24753.26,"low":24735.89}]
